## 네이버 카페 크롤링

In [1]:
# 라이브러리 불러오기
from selenium import webdriver # 브라우저 자동화를 위한 모듈
import requests # HTTP 요청을 보내기 위한 모듈
from bs4 import BeautifulSoup as BS # HTML 내용 파싱을 위한 모듈
import pandas as pd # 데이터 조작 및 분석을 모듈
import time # 코드 실행 속도 조절을 위한 모듈
import re # 정규 표현식 사용을 위한 모듈
import pickle # 파이썬 객체 직렬화를 위한 모듈
from selenium.webdriver.common.by import By # 다양한 방법으로 엘리먼트를 찾기 위한 모듈
import datetime # 날짜와 시간 연산을 위한 모듈

In [3]:
# 크롬 드라이버 파일 경로 설정
file_path = "C:\\Users\\cho03\\Downloads\\세미나\\방학 세미나\\"
path = file_path + 'chromedriver.exe'

# 크롬 드라이버 실행
driver = webdriver.Chrome(path)

C:\Users\cho03\AppData\Local\Temp\ipykernel_8604\388304711.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [4]:
# 네이버 카페 검색 페이지로 이동
driver.get('https://search.naver.com/search.naver?where=article&sm=tab_viw.blog&query=%EB%85%B8%EC%9D%B8%20%EB%B6%80%EC%96%91&nso=so%3Ar%2Cp%3Afrom20180705to20230705%2Ca%3Aall')

In [5]:
# 스코롤 다운 함수 정의
def doScrollDown(whileSeconds): 
    start = datetime.datetime.now() # 스크롤 다운 시작 시간 설정
    end = start + datetime.timedelta(seconds=whileSeconds) # 스크롤 다운 종료 시간 설정
    while True:
        # 페이지 맨 아래로 스크롤 다운
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1) # 1초 대기
        
        # 종료 시간에 도달하면 반복 종료
        if datetime.datetime.now() > end:  
            break

In [6]:
# 페이지 스크롤 다운
doScrollDown(101)

In [7]:
# 제목과 URL을 저장할 리스트 초기화
title_list = [] # 제목을 저장할 리스트 초기화
url_list = [] # URL을 저장할 리스트 초기화

# 카페 글 크롤링
for i in range(1,1100):
    try:
        # 카페 글 목록에서 제목 추출
        title = driver.find_element(By.XPATH,'//*[@id="_view_review_body_html"]/div/more-contents/div/ul/li[{}]/div[1]/div/a'.format(i)).text # 제목
        title2 = driver.find_element(By.XPATH,'//*[@id="_view_review_body_html"]/div/more-contents/div/ul/li[{}]/div[1]/div/a'.format(i))
        url = title2.get_attribute('href') # 카페 글 목록에서 URL 추출
    
        title_list.append(title) # 제목 추가
        url_list.append(url) # URL 주소 추가
    except:
        # 예외 발생 시 반복 종료
        break
    
    # 매 10번째 반복마다 진행 상황 출력
    if(i % 10 == 0):
        print(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [8]:
# 본문, 좋아요 수, 댓글 수, 댓글, 이미지 수, 영상 수를 저장할 리스트를 초기화
new_doc = [] # 본문을 저장할 리스트 초기화
like_cnt = [] # 좋아요 수를 저장할 리스트 초기화
comment_cnt = [] # 댓글 수를 저장할 리스트 초기화
comment_list = [] # 댓글을 저장할 리스트 초기화
img_cnt = [] # 이미지 수를 저장할 리스트 초기화
div_cnt = [] # 영상 수를 저장할 리스트 초기화

# 카페 글 크롤링
for i in range(len(url_list)):
    url_path = url_list[i] # URL 불러오기
    driver.switch_to.window(driver.window_handles[0]) # 첫 번째 탭으로 이동
    driver.execute_script("window.open('{}')".format(url_path)) # URL 실행
    driver.switch_to.window(driver.window_handles[1]) # 두 번째 탭으로 이동
    
    time.sleep(2) # 2초 대기
    
    iframes = driver.find_elements(By.TAG_NAME, 'iframe') #iframe 전환
    
    if len(iframes) > 0:
        try:
            # iframe 전환하여 본문과 정보 추출
            driver.switch_to.frame('cafe_main')
            html = driver.page_source # 현재 페이지의 HTML 코드를 가져와서 html 변수에 저장
            soup = BS(html, 'html.parser') # 댓글 영역의 HTML 내용을 분석

            try:
                # 본문 추출 (article_viewer 클래스를 찾아서 본문 텍스트를 가져옴)
                a = soup.find('div', class_='article_viewer').get_text()

            except:
                # 본문을 찾지 못한 경우
                a = 'null'

            try:
                # 좋아요 수 추출 (u_cnt _count 클래스를 찾아서 좋아요 수 텍스트를 가져옴)
                b = soup.find('em', class_='u_cnt _count').get_text()

            except:
                # 좋아요 수를 찾지 못한 경우
                b = 'null'

            try:
                # 댓글 수 추출 (strong num 클래스를 찾아서 댓글 수 텍스트를 가져옴)
                c = soup.find('strong', class_='num').get_text()

            except:    
                # 댓글 수를 찾지 못한 경우
                c = 'null'

            try:
                # 댓글 추출 (span text_comment 클래스를 찾아서 댓글을 가져옴)
                d = ''
                commentList = soup.find_all('span', class_='text_comment')
                for t in commentList:
                    d += t.get_text() + "\n"
                d += "\n"
            except:
                # 댓글을 찾지 못한 경우
                d = 'null'

            # 이미지 수 추출 (se-image-resource 클래스를 찾아서 이미지 수를 가져옴)
            e = len(soup.find_all('img', class_='se-image-resource'))            
            
            # 영상 수 추출 (pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed 클래스를 찾아서 영상 수를 가져옴)
            f = len(soup.find_all('div', class_='pzp-ui-dimmed pzp-dimmed pzp-pc__dimmed'))
           
            # 추출한 데이터를 각각의 리스트에 추가
            new_doc.append(a)
            like_cnt.append(b)
            comment_cnt.append(c)
            comment_list.append(d)
            img_cnt.append(e)
            div_cnt.append(f)
            driver.switch_to.default_content() # iframe에서 기본 컨텐츠로 전환
        
        except:     
            new_doc.append('') # 본문이 없을 경우 빈 문자열 추가
            like_cnt.append('') # 좋아요 수가 없을 경우 빈 문자열 추가
            comment_cnt.append('') # 댓글 수가 없을 경우 빈 문자열 추가
            comment_list.append('') # 댓글이 없을 경우 빈 문자열 추가
            img_cnt.append('') # 이미지 수가 없을 경우 빈 문자열 추가
            div_cnt.append('') # 영상 수가 없을 경우 빈 문자열 추가

    else:
        a = 'null' 
        
    driver.close()
    time.sleep(0.3) # 0.3초 대기
    
    # 매 10번째 반복마다 진행 상황 출력
    if(i%10) == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980


In [9]:
# 크롤링 데이터를 데이터프레임으로 변환
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['like'] = like_cnt
raw_data['comment_cnt'] = comment_cnt
raw_data['comment_list'] = comment_list
raw_data['img'] = img_cnt
raw_data['div'] = div_cnt
raw_data['ch'] = 'naver'
raw_data['ch2'] = 'cafe'

In [10]:
# 데이터프레임을 pickle 파일로 저장
f = open(file_path + "naver_cafe.pkl", "wb")
pickle.dump(raw_data,f)

f.close()

In [11]:
# 크롬 드라이버 종료
driver.quit()

In [12]:
# 저장된 pickle 파일을 불러옴
f = open(file_path + "naver_cafe.pkl", "rb")
temp_file = pickle.load(f)
f.close()

temp_file

,title,doc,like,comment_cnt,comment_list,img,div,ch,ch2
0,치매 노인 부양문제로 훈련소 면제 가능할까요?,━━━━━━━━━━━━━━━━━━━━ *카페규정:https://cafe.naver...,0,6,훈련소면제는 본인 질병만 가능합니다. 가사등의 사유로는 불가능 합니다.\n생계의 사...,0,0,naver,cafe
1,초저출산’ 지속되면 2070년엔 월급 42% 국민연금으로 내 노인 부...,광고. ⏰ 변호사 법률상담 할인 이벤트 진행 중! (기간한정 ~ 7/28) - ...,0,0,\n,1,0,naver,cafe
2,<노인이 노인을 부양하는 시대 도래에> 선제적으로 대비했다,\n\n\n\n\n\n\n\n\n작년 65세 이상 노인 인구는 ​919만 8480명...,14,21,은퇴준비 잘하셨네요 존경합니다~\n1970년대 유비무환을 귀에 달고 \n산 세대라서...,10,0,naver,cafe
3,여태껏 가정 안에서 연로한 어르신을 돌봤습니다. 그러나 맞벌이...,★오른쪽 상단의 첨부파일을 다운받아주세요. ★학사고에서는 학습자님이 레포트의 구조...,0,1,"80%정도 작성된 유료자료도 있습니다😎(서론, 본론, 참고문헌 구성)✨학사고 제휴 ...",0,0,naver,cafe
4,알코올 중독자를 가상하여 나타날 수 있는 문제를 문제중심기록/...,★오른쪽 상단의 첨부파일을 다운받아주세요. ★학사고에서는 학습자님이 레포트의 구조...,0,1,"80%정도 작성된 유료자료도 있습니다😎(서론, 본론, 참고문헌 구성)✨학사고 제휴 ...",0,0,naver,cafe
...,...,...,...,...,...,...,...,...,...
985,중도퇴사자 국세청 소득세 연말정산 프로그램 돌릴때 부양가족같...,\n\n\n\n\n\n\n\n\n============================...,0,4,나중에 부양가족이 빠짐으로써 환수될 수도 있어서 저는 중도퇴사자분 처리할 때 기존의...,0,0,naver,cafe
986,근로복지공단 대출,"베트남 나트랑,푸꾸옥 정보도 궁금하다면? [나트랑도깨비] https://cafe...",0,10,안녕하세요~^^\n반가워요\n안녕하세요 안녕하세요\n반가워요 안녕하세요\n안녕하세요...,2,0,naver,cafe
987,노인 기준 연령 상향 반대입안,\n\n\n\n\n\n\n\n\n안녕하세요. 노인 기준 연령을 상향해야 한다는 주제...,1,2,다음에 저희도 촉법소년 해요...이번에 어려운거 했으니까 좀 쉬어가는 느낌으로\nㅎ...,1,0,naver,cafe
988,자식들의 짐이 된 할머니(우울한 얘기 죄송합니다. ㅠㅠ),\n\n\n\n\n\n\n\n\n평생 자식들에게 당당하게 돈을 요구하신 할머니가 병...,0,0,\n,77,0,naver,cafe


In [13]:
# 데이터프레임을 CSV 파일로 저장
temp_file.to_csv("naver_cafe.csv", index = False, encoding="utf-8-sig")